## ERP SCANR


This notebook ...
.

In [2]:
## Imports

# Import custom code
from erpsc import *

# TEST IMPORTS
import requests
import nltk
from bs4 import BeautifulSoup

In [3]:
## Things to search through - full set
erps = ['P50', 'P100', 'P200', 'P300', 'P3a', 'P3b', 'P600', 'N100', 'N170', 'N200',\
         'N270', 'N2pc', 'N400', 'MMN', 'LPC', 'CNV', 'ERN', 'ELAN', 'CPS', 'LRP', 'LDN']
cogs = ['language', 'memory', 'attention', 'motor', 'decision making', 'vision', \
         'auditory', 'emotion', 'categorization', 'reward', 'spatial']

In [4]:
# Small test set of words
erps = ['P300', 'P600']
cogs = ['language', 'memory']

## Co-Occurence of Term Analysis

.

In [5]:
# Initialize object for term count co-occurences. 
term_counts = ERPSC_Count()

# Set erp and cog lists as terms to use
term_counts.set_erps(erps)
term_counts.set_cogs(cogs)

In [6]:
# Scrape the co-occurence of terms data
term_counts.scrape_data()

In [7]:
# Check the most commonly associated COG term for each ERP
term_counts.check_erps()

For the  P50   the most common association is 	 memory     with 	 %01.39
For the  P100  the most common association is 	 memory     with 	 %02.44


In [8]:
# Check the most commonly associated ERP term for each COG
term_counts.check_cogs()

For  language             the strongest associated ERP is 	 P100  with 	 %00.60
For  memory               the strongest associated ERP is 	 P100  with 	 %02.44


In [9]:
# Check the terms with the most papers
term_counts.check_top()

The most studied ERP is  P50     with     8177 papers
The most studied COG is  memory  with   213047  papers


In [10]:
# Check how many papers were found for each term - ERPs
term_counts.check_counts('erp')

P50   -     8177
P100  -     2171


In [11]:
# Check how many papers were found for each term - COGs
term_counts.check_counts('cog')

language           -     135605
memory             -     213047


In [12]:
# Save pickle file of results
term_counts.save_pickle()

In [13]:
# Load from pickle file
term_counts = load_pickle_counts()

## WORD ANALYSIS

.

In [4]:
# Initialize words-analysis object
words_analysis = ERPSC_Words()

In [5]:
# Set ERP and COG terms
words_analysis.set_erps(erps)
#words_analysis.set_cogs(cogs)

In [6]:
# Scrape word data for all ERP abstracts
words_analysis.scrape_data()

In [8]:
aa = words_analysis.results[0].words[0]

In [14]:
cc = nltk.FreqDist(aa)
type(cc)

nltk.probability.FreqDist

In [1]:
aa = words_analysis.results[0].words[0]

NameError: name 'words_anal' is not defined

In [9]:
FreqDist(aa)

FreqDist({u'(h3k18)': 1,
          u'(hat)': 1,
          u'3-fold': 1,
          u'a': 5,
          u'acetylated-lys': 1,
          u'acetylation': 5,
          u'acetyltransferase': 1,
          u'activates': 1,
          u'activation': 1,
          u'acute': 1,
          u'addition,': 1,
          u'allosterically': 1,
          u'an': 1,
          u'and': 10,
          u'anti-proliferative': 1,
          u'approach': 1,
          u'are': 1,
          u'as': 1,
          u'be': 2,
          u'been': 2,
          u'bind': 1,
          u'blots,': 1,
          u'both': 1,
          u'bromodomain': 3,
          u'but': 1,
          u'by': 3,
          u'can': 2,
          u'cancer': 2,
          u'cancer.': 1,
          u'cbp': 1,
          u'cbp30': 1,
          u'cbp30,': 1,
          u'cbp30.': 1,
          u'cells': 1,
          u'closely': 1,
          u'coactivators': 1,
          u'commensurate': 1,
          u'concentration': 1,
          u'conserved': 1,
          u'diseases.':

In [77]:
bb = aa.lower()
bb

u'the histone acetyltransferase (hat) enzymes p300 and cbp are closely related paralogs that serve as transcriptional coactivators and have been found to be dysregulated in cancer and other diseases. p300/cbp is a multi-domain protein and possesses a highly conserved bromodomain that has been shown to bind acetylated-lys residues in both proteins and various small molecules including i-cbp112 and cbp30. here we show that the ligand i-cbp112 can stimulate nucleosome acetylation up to 3-fold while cbp30 does not. activation of p300/cbp by i-cbp112 is not observed with isolated histone h3 substrate but requires a nucleosome substrate. i-cbp112 does not impact nucleosome acetylation by the isolated p300 hat domain and the effects of i-cbp112 on p300/cbp can be neutralized by cbp30, suggesting that i-cbp112 likely allosterically activates p300/cbp through bromodomain interactions. using mass spectrometry and western blots, we have found that i-cbp112 particularly stimulates acetylation of l

In [72]:
FreqDist(aa)

FreqDist({u'(H3K18)': 1,
          u'(HAT)': 1,
          u'3-fold': 1,
          u'Activation': 1,
          u'CBP': 1,
          u'CBP30': 1,
          u'CBP30,': 1,
          u'CBP30.': 1,
          u'H3': 2,
          u'H3K18': 1,
          u'HAT': 1,
          u'Here': 1,
          u'I-CBP112': 8,
          u'In': 1,
          u'Lys18': 1,
          u'Our': 1,
          u'The': 1,
          u'Using': 1,
          u'a': 5,
          u'acetylated-Lys': 1,
          u'acetylation': 5,
          u'acetyltransferase': 1,
          u'activates': 1,
          u'acute': 1,
          u'addition,': 1,
          u'allosterically': 1,
          u'an': 1,
          u'and': 10,
          u'anti-proliferative': 1,
          u'approach': 1,
          u'are': 1,
          u'as': 1,
          u'be': 2,
          u'been': 2,
          u'bind': 1,
          u'blots,': 1,
          u'both': 1,
          u'bromodomain': 3,
          u'but': 1,
          u'by': 3,
          u'can': 2,
          u'cancer

In [26]:
# Test Code

#page = requests.get('http://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&field=word&term=“N270”AND”Language”')
page = requests.get('http://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&field=word&term=“P300”&retmax=10')

page_soup = BeautifulSoup(page.content)

counts = page_soup.find_all('count')

for i in range(0, len(counts)):
    count = counts[i]
    ext = count.text
    print int(ext)

10087
10087


In [42]:
art_page = requests.get('http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&retmode=xml&id=' + id_strs)

art_page_soup = BeautifulSoup(art_page.content, "xml")